# Demo: Computing Relational Bias

This notebook demonstrates how to:
1. Load a pre-trained model using DeepNSD
2. Extract features from relation task stimuli
3. Compute distances between image embeddings
4. Calculate the relational bias metric

## About DeepJuice vs DeepNSD

**DeepJuice** is currently in private beta and provides a streamlined API for model feature extraction. If you're interested in using DeepJuice, please contact the authors to request access.

**DeepNSD** is the publicly available library that contains the core functionality of DeepJuice. For this demo, we use DeepNSD to ensure anyone can reproduce these results.

**DeepNSD Public Repo:** https://github.com/ColinConwell/DeepNSD

## 1. Setup and Installation

The following cell will automatically check for DeepNSD and install it if needed. No manual setup required!

In [34]:
# Setup DeepNSD Library

import sys
from pathlib import Path
import subprocess

# Check if DeepNSD is already available
deepnsd_path = Path("DeepNSD")

if not deepnsd_path.exists():
    print("DeepNSD not found. Installing...")
    print("=" * 60)
    
    # Clone the repository
    print("\n1. Cloning DeepNSD repository...")
    result = subprocess.run(
        ["git", "clone", "https://github.com/ColinConwell/DeepNSD.git"],
        capture_output=True,
        text=True
    )
    
    if result.returncode != 0:
        print(f"❌ Error cloning repository: {result.stderr}")
        raise Exception("Failed to clone DeepNSD")
    
    print("✓ Repository cloned successfully")
    
    # Install requirements
    print("\n2. Installing requirements...")
    requirements_file = deepnsd_path / "requirements.txt"
    if requirements_file.exists():
        result = subprocess.run(
            [sys.executable, "-m", "pip", "install", "-q", "-r", str(requirements_file)],
            capture_output=True,
            text=True
        )
        
        if result.returncode != 0:
            print(f"⚠ Warning: Some requirements may have failed to install")
            print(f"Error: {result.stderr}")
        else:
            print("✓ Requirements installed successfully")
    
    print("\n" + "=" * 60)
    print("✓ DeepNSD installation complete!\n")
else:
    print("✓ DeepNSD already installed")

# Add DeepNSD to Python path
if deepnsd_path.exists():
    # Insert at position 0 to ensure it takes precedence
    sys.path.insert(0, str(deepnsd_path.absolute()))
    print(f"✓ Added DeepNSD to path: {deepnsd_path.absolute()}")
else:
    print("❌ DeepNSD directory not found after installation")
    raise Exception("DeepNSD installation failed")

print("\n✓ Setup complete! Ready to use DeepNSD for feature extraction.")

✓ DeepNSD already installed
✓ Added DeepNSD to path: /user_data/wenjiel2/abstraction/submission_package/DeepNSD

✓ Setup complete! Ready to use DeepNSD for feature extraction.


In [35]:
# Import required libraries
import numpy as np
import pandas as pd
import torch
from pathlib import Path
from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
import os
import sys
import importlib.util

# Configure matplotlib for VS Code notebook display
matplotlib.use('module://matplotlib_inline.backend_inline')

# Load DeepNSD modules using importlib to handle relative imports
deepnsd_model_opts = Path(os.getcwd()) / "DeepNSD" / "source_code" / "pressures" / "model_opts"
if not deepnsd_model_opts.exists():
    raise FileNotFoundError(f"DeepNSD model_opts not found at {deepnsd_model_opts}")

print(f"✓ Found DeepNSD model_opts: {deepnsd_model_opts}")

# Step 1: Load model_options module first
model_options_path = deepnsd_model_opts / "model_options.py"
spec = importlib.util.spec_from_file_location("model_options", model_options_path)
model_options_module = importlib.util.module_from_spec(spec)
sys.modules['model_options'] = model_options_module
spec.loader.exec_module(model_options_module)
get_model_options = model_options_module.get_model_options

print("✓ Loaded model_options module")

# Step 2: Load feature_extraction module
# We need to inject the model_options contents to handle "from .model_options import *"
feature_extraction_path = deepnsd_model_opts / "feature_extraction.py"

# Read the source and replace the relative import
with open(feature_extraction_path, 'r') as f:
    source_code = f.read()

# Replace relative import with absolute import
source_code = source_code.replace('from .model_options import *', 'from model_options import *')

# Create and execute the modified module
spec = importlib.util.spec_from_file_location("feature_extraction", feature_extraction_path)
feature_extraction_module = importlib.util.module_from_spec(spec)
sys.modules['feature_extraction'] = feature_extraction_module

# Execute the modified code
exec(compile(source_code, feature_extraction_path, 'exec'), feature_extraction_module.__dict__)
get_all_feature_maps = feature_extraction_module.get_all_feature_maps

print("✓ Loaded feature_extraction module")

# Add code directory to path to import extraction functions
code_dir = Path(os.getcwd()) / "code" / "extraction"
if code_dir.exists():
    sys.path.insert(0, str(code_dir))
    print(f"✓ Added code directory to path: {code_dir}")
else:
    print(f"⚠ Warning: Code directory not found at {code_dir}")

# Import extraction functions from code directory
from core.model_processor import wrap_transform_with_resize
from core.metrics import compute_triplet_distances

print("✓ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Matplotlib backend: {matplotlib.get_backend()}")

✓ Found DeepNSD model_opts: /user_data/wenjiel2/abstraction/submission_package/DeepNSD/source_code/pressures/model_opts
✓ Loaded model_options module
✓ Loaded feature_extraction module
✓ Added code directory to path: /user_data/wenjiel2/abstraction/submission_package/code/extraction
✓ Imports successful
PyTorch version: 2.8.0+cu128
CUDA available: False
Matplotlib backend: module://matplotlib_inline.backend_inline


## 2. Load Relation Task Data

The relation task uses triplets of images:
- **Sample**: The reference image
- **Correct (Relational match)**: Shares the same abstract relation.
- **Incorrect (Area match)**: Shares the same number of green pixels but different relation.

In [36]:
# Set paths relative to notebook location
# This notebook should be run from the submission_package/ directory

import os

notebook_dir = Path(os.getcwd())

STIMULI_DIR = notebook_dir / "data/stimuli/relation"
PARQUET_FILE = notebook_dir / "data/model_performance/relation.parquet"

print(f"✓ Working directory: {notebook_dir}")
print(f"✓ Data directory verified")

# Load trial information
df = pd.read_parquet(PARQUET_FILE)

# Extract key columns
trial_info = df[['Sample', 'Correct', 'Incorrect', 'sample_idx', 'correct_idx', 'incorrect_idx']].copy()

print(f"\n✓ Loaded {len(trial_info)} trials")
print(f"\nFirst 5 trials:")
print(trial_info.head())

# Get all unique image files
all_images = sorted([str(f) for f in STIMULI_DIR.glob("*.png")])
print(f"\n✓ Found {len(all_images)} stimulus images")
print(f"Example images: {[Path(img).name for img in all_images[:3]]}")

✓ Working directory: /user_data/wenjiel2/abstraction/submission_package
✓ Data directory verified

✓ Loaded 126 trials

First 5 trials:
       Sample     Correct   Incorrect  sample_idx  correct_idx  incorrect_idx
0  SST_45.png  SST_65.png  STS_45.png           9           47             10
1  SST_45.png  SST_65.png  TST_45.png           9           47              0
2  SST_45.png  SST_65.png  TTT_45.png           9           47             32
3  SST_45.png  SST_85.png  SSS_45.png           9            4             28
4  SST_45.png  SST_85.png  STS_45.png           9            4             10

✓ Found 50 stimulus images
Example images: ['SSS_15.png', 'SSS_45.png', 'SST_35.png']


### Visualize Example Trial

In [37]:
# Visualize first trial
trial_idx = 0
sample_img = Image.open(STIMULI_DIR / trial_info.iloc[trial_idx]['Sample'])
correct_img = Image.open(STIMULI_DIR / trial_info.iloc[trial_idx]['Correct'])
incorrect_img = Image.open(STIMULI_DIR / trial_info.iloc[trial_idx]['Incorrect'])

fig, axes = plt.subplots(1, 3, figsize=(12, 4))
axes[0].imshow(sample_img)
axes[0].set_title(f"Sample\n{trial_info.iloc[trial_idx]['Sample']}", fontsize=10)
axes[0].axis('off')

axes[1].imshow(correct_img)
axes[1].set_title(f"Correct (Relational)\n{trial_info.iloc[trial_idx]['Correct']}", fontsize=10, color='green')
axes[1].axis('off')

axes[2].imshow(incorrect_img)
axes[2].set_title(f"Incorrect (Perceptual)\n{trial_info.iloc[trial_idx]['Incorrect']}", fontsize=10, color='red')
axes[2].axis('off')

plt.tight_layout()

# Save figure
output_file = 'example_trial.png'
fig.savefig(output_file, dpi=150, bbox_inches='tight')
plt.close(fig)

print(f"✓ Figure saved: {output_file}")
print(f"   View the image at: {Path(output_file).absolute()}")

✓ Figure saved: example_trial.png
   View the image at: /user_data/wenjiel2/abstraction/submission_package/example_trial.png


## 3. Load Model with DeepNSD

DeepNSD provides access to models from multiple sources:
- `torchvision_*` - TorchVision models
- `timm_*` - TIMM library models  
- `openclip_*` - OpenCLIP models

Models are specified by their source, architecture, dataset, and weights.

In [39]:
# Select a model to test
# 
# IMPORTANT: DeepNSD uses different naming than the stored results CSV
# 
# DeepNSD format: {model_name}_{train_type}
# CSV format: {source}_{model_name}_{dataset}_{version}
# 
# Example mappings:
# - DeepNSD: "resnet50_classification"  → CSV: "torchvision_resnet50_imagenet1k_v1"
# - DeepNSD: "alexnet_classification"   → CSV: "torchvision_alexnet_imagenet1k_v1"
# - DeepNSD: "vgg16_classification"     → CSV: "torchvision_vgg16_imagenet1k_v1"
#

model_uid = "resnet50_classification"  # Corresponds to torchvision_resnet50_imagenet1k_v1

# Map to CSV name for verification later
model_uid_csv = "torchvision_resnet50_imagenet1k_v1"

print(f"Loading model: {model_uid}")
print(f"(Corresponds to CSV model: {model_uid_csv})")

# Get model options and load model using DeepNSD
model_options = get_model_options()
if model_uid not in model_options:
    print(f"\n❌ Error: Model '{model_uid}' not found in model options")
    print(f"\nAvailable models (first 10):")
    for i, uid in enumerate(list(model_options.keys())[:10]):
        print(f"  {i+1}. {uid}")
    raise ValueError(f"Model {model_uid} not found in model options")

# Get the model loading function from model_options module
# This makes the necessary functions available for eval()
from model_options import get_torchvision_model, get_torchvision_transforms

# Execute the model call to get model and preprocessing
model_call = model_options[model_uid]['call']
model = eval(model_call)

# Get preprocessing transforms
train_type = model_options[model_uid]['train_type']
preprocess = get_torchvision_transforms(train_type, input_type='PIL')

# Move to GPU if available and set to eval mode
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.eval()
if device == 'cuda':
    model = model.cuda()

print(f"✓ Model loaded successfully on {device}")
print(f"Model type: {type(model).__name__}")

Loading model: resnet50_classification
(Corresponds to CSV model: torchvision_resnet50_imagenet1k_v1)
✓ Model loaded successfully on cpu
Model type: ResNet


## 4. Extract Features from Stimuli

We'll extract features from the penultimate layer (layer -2) of the model.

**Important:** We wrap the model's preprocessing with a resize operation to ensure all images are consistently 224x224 before being processed. This matches the pipeline methodology used to generate the stored results.

In [40]:
# Wrap preprocessing with resize to ensure consistent 224x224 images
# This matches the pipeline preprocessing used to generate stored results
wrapped_preprocess = wrap_transform_with_resize(preprocess, target_size=224)

# Create dataset and dataloader manually
from torch.utils.data import Dataset, DataLoader

class ImageDataset(Dataset):
    def __init__(self, image_paths, transform):
        self.image_paths = image_paths
        self.transform = transform
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img

dataset = ImageDataset(all_images, wrapped_preprocess)
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

print(f"✓ Wrapped preprocessing with resize to 224x224")
print(f"Created dataloader with batch size {batch_size}")
print(f"Total batches: {len(dataloader)}")
print(f"Total images: {len(dataset)}")

✓ Wrapped preprocessing with resize to 224x224
Created dataloader with batch size 16
Total batches: 4
Total images: 50


In [41]:
# Extract features using DeepNSD's get_all_feature_maps
# This returns a dictionary of layer_name -> features

print(f"Extracting features from all layers...")

# Extract all feature maps
feature_maps = get_all_feature_maps(
    model=model,
    inputs=dataloader,
    remove_duplicates=True,  # Remove duplicate layers
    flatten=True,            # Flatten spatial dimensions
    numpy=True,              # Return as numpy arrays
    use_tqdm=True            # Show progress bar
)

# Get layer names
layer_names = list(feature_maps.keys())
print(f"\n✓ Feature extraction complete!")
print(f"Total layers extracted: {len(layer_names)}")
print(f"\nLast 5 layers:")
for i, name in enumerate(layer_names[-5:]):
    print(f"  {len(layer_names) - 5 + i}: {name} - shape {feature_maps[name].shape}")

# Select the penultimate layer (layer -2)
# This is the layer before the final classification layer
penultimate_layer_name = layer_names[-2]
features = feature_maps[penultimate_layer_name]

print(f"\n✓ Selected penultimate layer: {penultimate_layer_name}")
print(f"Feature shape: {features.shape}")
print(f"  - {features.shape[0]} images")
print(f"  - {features.shape[1]} features per image")

Extracting features from all layers...


Feature Extraction (Batch):   0%|          | 0/4 [00:00<?, ?it/s]


✓ Feature extraction complete!
Total layers extracted: 158

Last 5 layers:
  153: Conv2d-53 - shape (50, 100352)
  154: BatchNorm2d-53 - shape (50, 100352)
  155: ReLU-49 - shape (50, 100352)
  156: AdaptiveAvgPool2d-1 - shape (50, 2048)
  157: Linear-1 - shape (50, 1000)

✓ Selected penultimate layer: AdaptiveAvgPool2d-1
Feature shape: (50, 2048)
  - 50 images
  - 2048 features per image


## 5. Compute Triplet Distances

For each trial, we compute:
- `dist_correct`: Cosine distance between sample and correct (relational) image
- `dist_incorrect`: Cosine distance between sample and incorrect (perceptual) image
- `distance_diff = dist_incorrect - dist_correct`

**Interpretation:**
- If `distance_diff > 0`: Model prefers relational match (correct)
- If `distance_diff < 0`: Model prefers area match (incorrect)
- If `distance_diff = 0`: Model is indifferent

We use the `compute_triplet_distances()` function from `code/extraction/core/metrics.py` to ensure we match the exact pipeline methodology.

In [42]:
# Create filename-based index mapping
# Since we're using sorted images, we need to map filenames to their indices
# in our sorted list (rather than using stored indices which were based on
# a different glob order from the pipeline)

import os

# Create mapping from filename to index in our sorted image list
filename_to_idx = {os.path.basename(img_path): idx 
                   for idx, img_path in enumerate(all_images)}

# Map trial filenames to indices in our sorted image list
sample_indices = trial_info['Sample'].apply(lambda x: filename_to_idx[x]).values
correct_indices = trial_info['Correct'].apply(lambda x: filename_to_idx[x]).values
incorrect_indices = trial_info['Incorrect'].apply(lambda x: filename_to_idx[x]).values

print(f"✓ Created filename-based index mapping")
print(f"  Example: {trial_info.iloc[0]['Sample']} -> index {sample_indices[0]}")

# Compute distances using imported function from code/extraction/core/metrics.py
distance_diffs = compute_triplet_distances(
    features,
    sample_indices,
    correct_indices,
    incorrect_indices,
    validate=False  # Skip validation for cleaner output
)

print(f"\n✓ Computed distances for {len(distance_diffs)} trials using pipeline function")
print(f"\nDistance difference statistics:")
print(f"  - Mean: {distance_diffs.mean():.6f}")
print(f"  - Std: {distance_diffs.std():.6f}")
print(f"  - Min: {distance_diffs.min():.6f}")
print(f"  - Max: {distance_diffs.max():.6f}")
print(f"  - Median: {np.median(distance_diffs):.6f}")

✓ Created filename-based index mapping
  Example: SST_45.png -> index 3

✓ Computed distances for 126 trials using pipeline function

Distance difference statistics:
  - Mean: -0.012868
  - Std: 0.030074
  - Min: -0.239551
  - Max: 0.048380
  - Median: -0.010973


### Visualize Distance Distribution

In [43]:
# Plot distribution of distance differences
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

ax.hist(distance_diffs, bins=30, edgecolor='black', alpha=0.7)
ax.axvline(0, color='red', linestyle='--', linewidth=2, label='Chance (0)')
ax.axvline(distance_diffs.mean(), color='blue', linestyle='--', linewidth=2, 
           label=f'Mean ({distance_diffs.mean():.4f})')

ax.set_xlabel('Distance Difference (dist_incorrect - dist_correct)', fontsize=12)
ax.set_ylabel('Number of Trials', fontsize=12)
ax.set_title(f'Distribution of Distance Differences\n{model_uid}', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()

# Save figure
output_file = 'distance_distribution.png'
fig.savefig(output_file, dpi=150, bbox_inches='tight')
plt.close(fig)

print(f"✓ Figure saved: {output_file}")
print(f"   View the image at: {Path(output_file).absolute()}")

print(f"\nTrials with positive difference (relational bias): {(distance_diffs > 0).sum()} / {len(distance_diffs)}")
print(f"Trials with negative difference (perceptual bias): {(distance_diffs < 0).sum()} / {len(distance_diffs)}")
print(f"Trials at zero: {(distance_diffs == 0).sum()} / {len(distance_diffs)}")

✓ Figure saved: distance_distribution.png
   View the image at: /user_data/wenjiel2/abstraction/submission_package/distance_distribution.png

Trials with positive difference (relational bias): 32 / 126
Trials with negative difference (perceptual bias): 94 / 126
Trials at zero: 0 / 126


## 6. Calculate Relational Bias

**Relational Bias** is the proportion of trials where the model prefers the relational match over the perceptual match.

$$\text{Relational Bias} = \frac{\text{# trials with } (\text{dist_incorrect} - \text{dist_correct}) > 0}{\text{Total trials}}$$

- **Relational Bias > 0.5**: Model shows relational bias (prefers abstract relations)
- **Relational Bias < 0.5**: Model shows perceptual bias (prefers superficial features)
- **Relational Bias = 0.5**: Model is at chance (no preference)

In [44]:
def compute_relational_bias(distance_diffs):
    """
    Compute relational bias as proportion of trials where model prefers
    the relational match (distance_diff > 0).
    
    Args:
        distance_diffs: Array of distance differences
    
    Returns:
        Relational bias (proportion between 0 and 1)
    """
    # Filter out any NaN values
    valid_diffs = distance_diffs[~np.isnan(distance_diffs)]
    
    if len(valid_diffs) == 0:
        return np.nan
    
    # Proportion where distance_diff > 0 (model prefers relational match)
    relational_bias = (valid_diffs > 0).mean()
    
    return float(relational_bias)

# Calculate relational bias
relational_bias = compute_relational_bias(distance_diffs)

print("="*60)
print(f"RELATIONAL BIAS: {relational_bias:.4f}")
print("="*60)

# Interpretation
if relational_bias > 0.58:  # Binomial test threshold for 126 trials, α=0.05
    interpretation = "🟢 RELATIONAL BIAS (significantly above chance)"
elif relational_bias < 0.42:
    interpretation = "🔴 AREA BIAS (significantly below chance)"
else:
    interpretation = "⚪ AT CHANCE (no clear preference)"

print(f"\nInterpretation: {interpretation}")
print(f"\nModel: {model_uid}")
print(f"Total trials: {len(distance_diffs)}")
print(f"Trials preferring relational match: {(distance_diffs > 0).sum()}")
print(f"Trials preferring area match: {(distance_diffs < 0).sum()}")
print(f"\nNote: Chance level is 0.5")
print(f"Binomial test threshold (126 trials, α=0.05, one-sided):")
print(f"  - Relational bias: ≥ 0.5794")
print(f"  - Area bias: ≤ 0.4206")

RELATIONAL BIAS: 0.2540

Interpretation: 🔴 AREA BIAS (significantly below chance)

Model: resnet50_classification
Total trials: 126
Trials preferring relational match: 32
Trials preferring area match: 94

Note: Chance level is 0.5
Binomial test threshold (126 trials, α=0.05, one-sided):
  - Relational bias: ≥ 0.5794
  - Area bias: ≤ 0.4206


## 7. Compare with Human Performance

The parquet file includes human behavioral data from different groups.

In [45]:
# Extract human performance (averaged across all trials)
human_cols = ['USADULT_Accuracy', 'TSIADULT_Accuracy', 'KID_Accuracy', 'MONKEY_Accuracy']

print("Human and Monkey Performance:")
print("="*40)
print("(Averaged across all available trials)\n")

available_data = False
for col in human_cols:
    if col in df.columns:
        values = df[col].dropna()  # Remove NaN values
        if len(values) > 0:
            group = col.replace('_Accuracy', '')
            mean_accuracy = values.mean()
            n_trials = len(values)
            print(f"{group:15s}: {mean_accuracy:.4f} (n={n_trials} trials)")
            available_data = True

if not available_data:
    print("(No human data available)")

print("\nModel Performance:")
print("="*40)
print(f"{model_uid:15s}: {relational_bias:.4f} (n={len(distance_diffs)} trials)")

Human and Monkey Performance:
(Averaged across all available trials)

USADULT        : 0.9324 (n=74 trials)
TSIADULT       : 0.6633 (n=15 trials)
KID            : 0.8150 (n=30 trials)
MONKEY         : 0.4128 (n=20 trials)

Model Performance:
resnet50_classification: 0.2540 (n=126 trials)
